In [1]:
import importcsv

print('Importing csv file...........')
small_dataset = importcsv.opencsv('processedspeech.csv') 

Importing csv file...........


In [2]:
#Import the json file with the inverted catalog
import json

file_path = "inverted_catalog.json"

# Load the dictionary from the file
with open(file_path, 'r') as file:
    loaded_dictionary = json.load(file)

In [105]:
# the querry
from wordtools import process_string

querry = 'αποχή'

querry = process_string(querry).lower()

print(querry)

αποχ


In [87]:
#split the querry into words and check if they are indexed in the inverted catalog. return the indexes of the speeches that contain the querry words
querrylist = querry.split(' ')
indices = []
for word in querrylist:
    if word in loaded_dictionary.keys() and word not in indices:
        indices = indices + loaded_dictionary[word]
    else:
        print('Maybe use the deep search')
indices 

[33595,
 35968,
 53494,
 55533,
 59346,
 62349,
 70416,
 85146,
 97245,
 126163,
 129151,
 199442,
 264855]

In [88]:
#make a list from the indexes 
smaller_list = small_dataset['processed_speeches'].iloc[indices].to_list()


In [90]:
# Add as last row the querr. (Small note: the inverted catalog has the words in lower but the processed speeches are in upper)
smaller_list.append(querry.upper())


In [92]:
# tfid vectorizer as usual 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(smaller_list)

In [95]:
#cosine similarity
ddsim_matrix = cosine_similarity(tfidf_matrix[len(smaller_list)-1,:], tfidf_matrix)

In [96]:
print(ddsim_matrix)

[[0.05538282 0.         0.01379115 0.00906318 0.         0.0040644
  0.         0.         0.01341451 0.         0.         0.
  0.         1.        ]]


In [99]:
#remove assosiations less than 0
similar = {}
for i in range(len(smaller_list)-1):
    if ddsim_matrix[0,i] > 0:
        similar[i] = ddsim_matrix[0,i]

In [100]:
similar

{0: 0.055382818043697185,
 2: 0.013791153401994419,
 3: 0.009063175100387955,
 5: 0.004064397620628499,
 8: 0.013414506085353077}

In [101]:
#Find the top k similarity amongst the speeches based on querry
from heapq import nlargest

k = 80
top_k_items = nlargest(k, similar.items(), key=lambda x: x[1])

In [102]:
#print the top 5 speeches related to the querry

for i,list in enumerate(top_k_items):    
    print('Similar speech:', i)
    print('speech:',small_dataset.at[indices[list[0]],'speech'])
    #print(dataset3.at[speechstr,'processed1'])
    print('member_name:',small_dataset.at[indices[list[0]],'member_name'])
    print('political_party:',small_dataset.at[indices[list[0]],'political_party'])
    print('sitting_date:',small_dataset.at[indices[list[0]],'sitting_date'])
    if i == 6:
        break

Similar speech: 0
speech:  Κύριοι συνάδελφοι, αρχίζει η συνεδρίαση.Παρακαλείται ο κύριος Γραμματέας να διαβάσει τις αναφορές προς το Σώμα.Ανακοινώνονται προς το Σώμα, από τον υπάλληλο της Βουλής κ. Γεώργιο Μπιρμπίλη, τα ακόλουθα:Α` ΚΑΤΑΘΕΣΗ ΑΝΑΦΟΡΩΝ1. Ο Βουλευτής Κοζάνης κ. ΘΕΟΔΩΡΟΣ ΚΟΚΕΛΙΔΗΣ κατέθεσε αναφορές με τις οποίες το "Κέντρο Ερεύνης και Μελέτης Ελληνισμού  " ζητεί από τη Βουλή των Ελλήνων να αναγνωρίσει τη γενοκτονία των Ελλήνων της Μικράς Ασίας και του Πόντου, καθώς και τη γενοκτονία των Αρμενίων και όλων αυτών από τους Τούρκους.2.Ο Βουλευτής Χίου κ. ΑΝΤΩΝΗΣ ΚΟΤΣΑΚΑΣ κατέθεσε αναφορά με την οποία ο Μορφωτικός Σύνδεσμος Βαρβασίου Χίου ζητεί οικονομική ενίσχυση για τη διοργάνωση εκδηλώσεων. 3. Ο Βουλευτής Χίου κ. ΑΝΤΩΝΗΣ ΚΟΤΣΑΚΑΣ κατέθεσε αναφορά με την οποία ο Κυνηγετικός Σύλλογος Νομού Χίου "Η ΑΡΤΕΜΙΣ" ζητεί τη λήψη μέτρων για την προστασία των θηραμάτων στη νήσο Χίο. 4. Ο Βουλευτής Αιτωλ/νίας κ. ΧΡΙΣΤΟΣ ΚΟΚΚΙΝΟΒΑΣΙΛΗΣ κατέθεσε αναφορά με την οποία ο Πρόεδρος της Κοινότητας 